In [33]:
import pandas as pd
import numpy as np
import torch
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('data/symptoms_prompts_generated_data.csv')
print(df.shape)
df.head(3)

(9130, 23)


,user_prompt,thal,gluc,ap_lo,ap_hi,active,st_slope,maxhr,smoke,restingbp,...,cholesterol,fastingbs,exerciseangina,restingecg,age,alco,cheastpaintype,nummajorvessels,weight,height
0,"Я постоянно ощущаю сильное давление в груди, о...",1.0,1.0,131.0,226.0,1.0,1.0,154.0,0.0,79.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Меня беспокоит очень высокое давление в покое ...,2.0,0.0,NaN,NaN,NaN,NaN,75.0,NaN,195.0,...,1.0,0.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
2,"Мне 56 лет, жалуюсь на постоянную усталость и ...",NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,56.0,NaN,NaN,NaN,NaN,NaN


In [4]:
# разделим фичи на категориальные и вещественные
numerical_features = ['age', 'restingbp', 'ap_hi', 'weight', 'cholesterol_int', 'height', 'maxhr', 'ap_lo']
categorical_features = ['gluc', 'restingecg', 'fastingbs', 'cholesterol',
            'smoke', 'oldpeak', 'gender', 'thal', 'exerciseangina',
            'nummajorvessels', 'active', 'cheastpaintype', 'alco', 'st_slope']

In [5]:
# разделим датасет на трейн и тест (тест пригодится нам в конце для оценки модели)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [6]:
print(train_df.shape)
train_df.notna().sum() / train_df.shape[0]

(7304, 23)


,0
user_prompt,1.000000
thal,0.375411
gluc,0.380887
ap_lo,0.383762
ap_hi,0.368154
active,0.376643
st_slope,0.379107
maxhr,0.372673
smoke,0.371303
restingbp,0.376095


`В среднем 67% пропусков в данных, что нормально, ибо каждый пользователь в своем промте не может описать все симптомы (может просто не знать про них)`

In [7]:
numerical_scalers = {}
for feature in numerical_features:
    if feature in train_df.columns:
        values = train_df[feature].dropna().values
        if len(values) > 0:
            mean = values.mean()
            std = values.std() if values.std() > 0 else 1.0

            numerical_scalers[feature] = {"mean": mean, "std": std}
            print(f"Scaling {feature}: mean={mean:.2f}, std={std:.2f}")

Scaling age: mean=58.82, std=24.12
Scaling restingbp: mean=148.41, std=46.37
Scaling ap_hi: mean=150.32, std=52.51
Scaling weight: mean=117.82, std=46.32
Scaling cholesterol_int: mean=343.50, std=127.68
Scaling height: mean=185.69, std=20.13
Scaling maxhr: mean=154.69, std=49.20
Scaling ap_lo: mean=94.87, std=31.18


In [8]:
categorical_num_classes = {}
for feature in categorical_features:
    if feature in df.columns:
        unique_values = df[feature].unique()
        non_nan_values = [v for v in unique_values if not pd.isna(v)]
        num_classes = len(non_nan_values) + 1
        categorical_num_classes[feature] = num_classes
        print(f"{feature}: найдены значения {non_nan_values}, добавляем NaN -> всего {num_classes} классов")

gluc: найдены значения [np.float64(1.0), np.float64(0.0)], добавляем NaN -> всего 3 классов
restingecg: найдены значения [np.float64(2.0), np.float64(1.0), np.float64(0.0)], добавляем NaN -> всего 4 классов
fastingbs: найдены значения [np.float64(0.0), np.float64(1.0)], добавляем NaN -> всего 3 классов
cholesterol: найдены значения [np.float64(0.0), np.float64(1.0)], добавляем NaN -> всего 3 классов
smoke: найдены значения [np.float64(0.0), np.float64(1.0)], добавляем NaN -> всего 3 классов
oldpeak: найдены значения [np.float64(0.0), np.float64(1.0)], добавляем NaN -> всего 3 классов
gender: найдены значения [np.float64(0.0), np.float64(1.0)], добавляем NaN -> всего 3 классов
thal: найдены значения [np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(0.0)], добавляем NaN -> всего 5 классов
exerciseangina: найдены значения [np.float64(0.0), np.float64(1.0)], добавляем NaN -> всего 3 классов
nummajorvessels: найдены значения [np.float64(1.0), np.float64(3.0), np.float64(4.0), n

In [10]:
# Создаем маппинг для каждого категориального признака
categorical_mapping = {}
for feature in categorical_features:
    if feature in df.columns:
        unique_values = sorted(df[feature].dropna().unique())
        categorical_mapping[feature] = list(unique_values) + ["NaN"]
        print(f"{feature} mapping: {categorical_mapping[feature]}")

gluc mapping: [np.float64(0.0), np.float64(1.0), 'NaN']
restingecg mapping: [np.float64(0.0), np.float64(1.0), np.float64(2.0), 'NaN']
fastingbs mapping: [np.float64(0.0), np.float64(1.0), 'NaN']
cholesterol mapping: [np.float64(0.0), np.float64(1.0), 'NaN']
smoke mapping: [np.float64(0.0), np.float64(1.0), 'NaN']
oldpeak mapping: [np.float64(0.0), np.float64(1.0), 'NaN']
gender mapping: [np.float64(0.0), np.float64(1.0), 'NaN']
thal mapping: [np.float64(0.0), np.float64(1.0), np.float64(2.0), np.float64(3.0), 'NaN']
exerciseangina mapping: [np.float64(0.0), np.float64(1.0), 'NaN']
nummajorvessels mapping: [np.float64(0.0), np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0), 'NaN']
active mapping: [np.float64(0.0), np.float64(1.0), 'NaN']
cheastpaintype mapping: [np.float64(0.0), np.float64(1.0), np.float64(2.0), np.float64(3.0), 'NaN']
alco mapping: [np.float64(0.0), np.float64(1.0), 'NaN']
st_slope mapping: [np.float64(0.0), np.float64(1.0), np.float64(2.0), 'NaN']


In [14]:
from transformers import AutoTokenizer, AutoModel
from torch import nn

class SymptomExtractionModel(nn.Module):
    def __init__(self, pretrained_model_name="DeepPavlov/rubert-base-cased",
                 numerical_features=None, categorical_features=None,
                 categorical_num_classes=None):
        super(SymptomExtractionModel, self).__init__()

        self.bert = AutoModel.from_pretrained(pretrained_model_name)
        self.dropout = nn.Dropout(0.1)
        hidden_size = self.bert.config.hidden_size

        self.numerical_features = numerical_features or []
        self.numerical_heads = nn.ModuleDict()
        self.numerical_missing_heads = nn.ModuleDict()
        for feature in self.numerical_features:
            self.numerical_heads[feature] = nn.Sequential(
                nn.Linear(hidden_size, 128),
                nn.ReLU(),
                nn.Dropout(0.1),
                nn.Linear(128, 1)
            )
            self.numerical_missing_heads[feature] = nn.Linear(hidden_size, 2)  # 0 = значение есть, 1 = пропуск


        self.categorical_features = categorical_features or []
        self.categorical_heads = nn.ModuleDict()
        for feature in self.categorical_features:
            num_classes = categorical_num_classes.get(feature, 2)
            print(f"Создание классификационной головы для {feature} с {num_classes} классами (включая пропуск)")
            self.categorical_heads[feature] = nn.Linear(hidden_size, num_classes)

    def forward(self, input_ids, attention_mask, token_type_ids=None):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        sequence_output = outputs.last_hidden_state[:, 0, :]
        sequence_output = self.dropout(sequence_output)

        numerical_outputs = {
            feature: head(sequence_output) for feature, head in self.numerical_heads.items()
        }

        numerical_missing_outputs = {
            feature: head(sequence_output) for feature, head in self.numerical_missing_heads.items()
        }

        categorical_outputs = {
            feature: head(sequence_output) for feature, head in self.categorical_heads.items()
        }

        return numerical_outputs, numerical_missing_outputs, categorical_outputs

In [15]:
from torch.utils.data import Dataset, DataLoader

class SymptomsDataset(Dataset):
    def __init__(self, texts, numerical_features, categorical_features,
                 feature_values, tokenizer, categorical_mapping=None, max_length=512):
        self.texts = texts
        self.numerical_features = numerical_features
        self.categorical_features = categorical_features
        self.feature_values = feature_values
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.categorical_mapping = categorical_mapping or {}
        self.numerical_scalers = {}
        for feature in self.numerical_scalers:
            if feature in train_df.columns:
                # Находим непустые значения
                values = train_df[feature].dropna().values
                if len(values) > 0:
                    # Вычисляем среднее и стандартное отклонение
                    mean = values.mean()
                    std = values.std() if values.std() > 0 else 1.0

                    # Запоминаем параметры для каждого признака
                    numerical_scalers[feature] = {"mean": mean, "std": std}
                    print(f"Scaling {feature}: mean={mean:.2f}, std={std:.2f}")


    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )
        encoding = {k: v.squeeze(0) for k, v in encoding.items()}

        numerical_labels = {}
        numerical_missing_labels = {}
        for feature in self.numerical_features:
            value = self.feature_values[feature].iloc[idx]
            if pd.isna(value):
                numerical_missing_labels[feature] = torch.tensor([1], dtype=torch.long)  # 1 = пропуск
                numerical_labels[feature] = torch.tensor([0.0], dtype=torch.float)
            else:
                numerical_missing_labels[feature] = torch.tensor([0], dtype=torch.long)  # 0 = не пропуск
                numerical_labels[feature] = torch.tensor([float(value)], dtype=torch.float)

        categorical_labels = {}
        for feature in self.categorical_features:
            value = self.feature_values[feature].iloc[idx]
            if feature in self.categorical_mapping:
                if pd.isna(value):
                    class_idx = len(self.categorical_mapping[feature]) - 1
                else:
                    class_idx = self.categorical_mapping[feature].index(value)
                categorical_labels[feature] = torch.tensor([class_idx], dtype=torch.long)
            else:
                if pd.isna(value):
                    class_idx = self.feature_values[feature].dropna().nunique()
                    categorical_labels[feature] = torch.tensor([class_idx], dtype=torch.long)
                else:
                    categorical_labels[feature] = torch.tensor([int(value)], dtype=torch.long)
        
        for feature in self.numerical_features:
            value = self.feature_values[feature].iloc[idx]
            if pd.isna(value):
                numerical_missing_labels[feature] = torch.tensor([1], dtype=torch.long)
                numerical_labels[feature] = torch.tensor([0.0], dtype=torch.float)
            else:
                if feature in self.numerical_scalers:
                    normalized_value = (float(value) - self.numerical_scalers[feature]["mean"]) / self.numerical_scalers[feature]["std"]
                else:
                    normalized_value = float(value)

                numerical_missing_labels[feature] = torch.tensor([0], dtype=torch.long)
                numerical_labels[feature] = torch.tensor([normalized_value], dtype=torch.float)

        return {
            **encoding,
            "numerical_labels": numerical_labels,
            "numerical_missing_labels": numerical_missing_labels,
            "categorical_labels": categorical_labels
        }


In [17]:
import torch.optim as optim
from tqdm import tqdm
import sklearn.metrics as metrics

def train_model(model, train_dataloader, val_dataloader, device,
                numerical_features, categorical_features,
                categorical_mapping, epochs=5):
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    mse_loss = nn.L1Loss()
    ce_loss = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        train_samples = 0

        for batch_idx, batch in enumerate(train_dataloader):
            try:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                token_type_ids = batch.get('token_type_ids')
                if token_type_ids is not None:
                    token_type_ids = token_type_ids.to(device)

                numerical_outputs, numerical_missing_outputs, categorical_outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids
                )

                numerical_value_loss = 0.0
                numerical_missing_loss = 0.0
                num_numerical_samples = 0

                for feature in numerical_features:
                    if feature not in batch['numerical_labels']:
                        continue
                    missing_labels = batch['numerical_missing_labels'][feature].to(device).view(-1)
                    value_labels = batch['numerical_labels'][feature].to(device)
                    predictions = numerical_outputs[feature]
                    if torch.isnan(predictions).any() or torch.isinf(predictions).any():
                        print(f"Warning: NaN or Inf detected in predictions for {feature}")
                        predictions = torch.nan_to_num(predictions, nan=0.0, posinf=1e5, neginf=-1e5)

                    if torch.isnan(value_labels).any() or torch.isinf(value_labels).any():
                        print(f"Warning: NaN or Inf detected in labels for {feature}")
                        value_labels = torch.nan_to_num(value_labels, nan=0.0, posinf=1e5, neginf=-1e5)

                    feature_missing_loss = ce_loss(numerical_missing_outputs[feature], missing_labels)
                    numerical_missing_loss += feature_missing_loss
                    mask = (missing_labels == 0)  # Только непропущенные значения

                    if mask.sum() > 0:
                        clipped_predictions = torch.clamp(predictions[mask], -10.0, 10.0)
                        clipped_labels = torch.clamp(value_labels[mask], -10.0, 10.0)

                        feature_value_loss = mse_loss(clipped_predictions, clipped_labels)
                        numerical_value_loss += feature_value_loss

                categorical_loss = 0.0
                num_categorical_samples = 0

                for feature in categorical_features:
                    if feature not in batch['categorical_labels']:
                        continue

                    labels = batch['categorical_labels'][feature].to(device).view(-1)
                    outputs = categorical_outputs[feature]

                    feature_loss = ce_loss(outputs, labels)
                    categorical_loss += feature_loss
                    num_categorical_samples += 1

                if num_categorical_samples > 0:
                    categorical_loss /= num_categorical_samples

                # в итоговом лоссе зададим веса
                alpha = 1.0
                beta = 10.0
                gamma = 10.0
                total_loss = alpha * numerical_value_loss + beta * numerical_missing_loss + gamma * categorical_loss

                optimizer.zero_grad()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                total_loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Предотвращаем взрыв градиентов
                optimizer.step()
                train_loss += total_loss.item()
                train_samples += 1

                if batch_idx % 10 == 0:
                    print(f"Batch {batch_idx}, Loss: {total_loss.item():.4f} " +
                          f"(Val: {numerical_value_loss.item():.4f}, " +
                          f"Miss: {numerical_missing_loss.item():.4f}, " +
                          f"Cat: {categorical_loss.item():.4f})")

            except Exception as e:
                print(f"Error in batch {batch_idx}: {e}")
                import traceback
                traceback.print_exc()
                continue

        avg_train_loss = train_loss / max(1, train_samples)
        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {avg_train_loss:.4f}")

    return model

In [18]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")


train_dataset = SymptomsDataset(
    texts=train_df['user_prompt'].tolist(),
    numerical_features=numerical_features,
    categorical_features=categorical_features,
    feature_values=train_df,
    tokenizer=tokenizer,
    categorical_mapping=categorical_mapping
)

test_dataset = SymptomsDataset(
    texts=test_df['user_prompt'].tolist(),
    numerical_features=numerical_features,
    categorical_features=categorical_features,
    feature_values=test_df,
    tokenizer=tokenizer,
    categorical_mapping=categorical_mapping
)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(test_dataset, batch_size=16)

numerical_ranges = {
    'age': (18.0, 100.0),
    'restingbp': (70.0, 229.0),
    'ap_hi': (60.0, 239.0),
    'weight': (40.0, 200.0),
    'cholesterol_int': (126.0, 564.0),
    'height': (150.0, 220.0),
    'maxhr': (70.0, 239.0),
    'ap_lo': (40.0, 149.0)
}


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SymptomExtractionModel(
    numerical_features=numerical_features,
    categorical_features=categorical_features,
    categorical_num_classes={f: len(categorical_mapping[f]) for f in categorical_mapping}
)

trained_model = train_model(
    model=model,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    device=device,
    numerical_features=numerical_features,
    categorical_features=categorical_features,
    categorical_mapping=categorical_mapping,
    epochs=10
)

torch.save(trained_model.state_dict(), 'symptom_extraction_model_try_1.pt')

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Создание классификационной головы для gluc с 3 классами (включая пропуск)
Создание классификационной головы для restingecg с 4 классами (включая пропуск)
Создание классификационной головы для fastingbs с 3 классами (включая пропуск)
Создание классификационной головы для cholesterol с 3 классами (включая пропуск)
Создание классификационной головы для smoke с 3 классами (включая пропуск)
Создание классификационной головы для oldpeak с 3 классами (включая пропуск)
Создание классификационной головы для gender с 3 классами (включая пропуск)
Создание классификационной головы для thal с 5 классами (включая пропуск)
Создание классификационной головы для exerciseangina с 3 классами (включая пропуск)
Создание классификационной головы для nummajorvessels с 6 классами (включая пропуск)
Создание классификационной головы для active с 3 классами (включая пропуск)
Создание классификационной головы для cheastpaintype с 5 классами (включая пропуск)
Создание классификационной головы для alco с 3 классами

In [19]:
def predict_symptoms(text, model):
    """
    Предсказывает симптомы из текстового описания
    Args:
        text: текстовое описание симптомов пациента
    Returns:
        словарь с предсказанными симптомами
    """
    encoding = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"
    )
    encoding = {k: v.to(device) for k, v in encoding.items()}
    with torch.no_grad():
        numerical_outputs, numerical_missing_outputs, categorical_outputs = model(**encoding)

    predictions = {}
    for feature in numerical_features:
        missing_probs = torch.softmax(numerical_missing_outputs[feature], dim=1)
        is_missing = missing_probs[0, 1] > 0.5  # Вероятность пропуска > 0.5

        if is_missing:
            predictions[feature] = float('nan')
        else:
            value = numerical_outputs[feature].cpu().numpy()[0][0]

            # Денормализуем, если есть скейлер
            if feature in numerical_scalers:
                mean = numerical_scalers[feature]['mean']
                std = numerical_scalers[feature]['std']
                value = value * std + mean

            # Ограничиваем значение диапазоном
            if feature in numerical_ranges:
                min_val, max_val = numerical_ranges[feature]
                value = max(min_val, min(max_val, value))

            # Округляем до одного знака после запятой
            predictions[feature] = round(value, 1)

    for feature in categorical_features:
        probabilities = torch.softmax(categorical_outputs[feature], dim=1)
        predicted_class_idx = torch.argmax(probabilities, dim=1).cpu().numpy()[0]
        if feature in categorical_mapping:
            class_values = categorical_mapping[feature]
            if predicted_class_idx == len(class_values) - 1:  # Последний класс = пропуск
                predictions[feature] = float('nan')
            else:
                try:
                    predictions[feature] = float(class_values[predicted_class_idx])
                except:
                    predictions[feature] = class_values[predicted_class_idx]
        else:
            num_classes = probabilities.size(1)
            if predicted_class_idx == num_classes - 1:  # Последний класс
                predictions[feature] = float('nan')
            else:
                predictions[feature] = float(predicted_class_idx)
    return predictions

In [21]:
# загрузим обученную модель
model_state_dict = torch.load('symptom_extraction_model_try_1.pt', map_location=device)

predict_model = SymptomExtractionModel(
        numerical_features=numerical_features,
        categorical_features=categorical_features,
        categorical_num_classes=categorical_num_classes
    )
predict_model.load_state_dict(model_state_dict)
predict_model.to(device)
predict_model.eval()
from IPython.display import clear_output
clear_output()

In [26]:
# тестовый пример

test_text = "я женщина 32 года, постоянно испытываю дискомфорт в груди при физических нагрузках. Давление обычно повышенное"
predictions = predict_symptoms(test_text, model=predict_model)

print("\nПредсказанные симптомы:")
for feature, value in predictions.items():
    if pd.isna(value):
        print(f"{feature}: НЕТ ДАННЫХ (пропуск)")
    else:
        print(f"{feature}: {value}")


Предсказанные симптомы:
age: 100.0
restingbp: НЕТ ДАННЫХ (пропуск)
ap_hi: 239.0
weight: НЕТ ДАННЫХ (пропуск)
cholesterol_int: НЕТ ДАННЫХ (пропуск)
height: НЕТ ДАННЫХ (пропуск)
maxhr: НЕТ ДАННЫХ (пропуск)
ap_lo: 149.0
gluc: НЕТ ДАННЫХ (пропуск)
restingecg: НЕТ ДАННЫХ (пропуск)
fastingbs: НЕТ ДАННЫХ (пропуск)
cholesterol: НЕТ ДАННЫХ (пропуск)
smoke: НЕТ ДАННЫХ (пропуск)
oldpeak: НЕТ ДАННЫХ (пропуск)
gender: 0.0
thal: НЕТ ДАННЫХ (пропуск)
exerciseangina: НЕТ ДАННЫХ (пропуск)
nummajorvessels: НЕТ ДАННЫХ (пропуск)
active: НЕТ ДАННЫХ (пропуск)
cheastpaintype: НЕТ ДАННЫХ (пропуск)
alco: НЕТ ДАННЫХ (пропуск)
st_slope: НЕТ ДАННЫХ (пропуск)


In [44]:
# оценим качество модели на тестовой выборке - аля "accuracy" - доля правильно угаданных симптомов
correct, total = 0, 0
for i, row in tqdm(test_df.iterrows()):
  prompt = row['user_prompt']
  prediction = predict_symptoms(prompt, predict_model)
  for feature in numerical_features:
    if pd.isna(row[feature]) and pd.isna(prediction[feature]):
      correct += 1
      total += 1
    elif row[feature] == prediction[feature]:
      correct += 1
      total += 1
    else:
      total += 1
  for feature in categorical_features:
    if pd.isna(row[feature]) and pd.isna(prediction[feature]):
      correct += 1
      total += 1
    elif row[feature] == prediction[feature]:
      correct += 1
      total += 1
    else:
      total += 1

print(f'TEST ACCURACY = {round(correct / total * 100, 1)}%')

1826it [01:17, 23.42it/s]

TEST ACCURACY = 64.5%


#### **Выводы**

С помощью использования предобученной модели *DeepPavlov/rubert-base-cased*, а также дополнительного дообучения модели в 10 эпох получили неплохую итоговую точность - 64.5%.

Возможные шаги по ее улучшению:

1. Попробовать другие предобученные модели

2. Усложнить выходную архитектуру (более сложные "головы")

3. Увеличить количество эпох во время обучения (требует больше ресурсов)

4. Подобрать гиперпараметры (в данном случае "пороги пропусков", "веса при лоссах")